## ДЗ №3 Двухуровневый пайплайн
#### В этой домашке вам предстоит написать с нуля двустадийную рекомендательную систему. 

#### Дата выдачи: 10.03.25

#### Мягкий дедлайн: 31.03.25 23:59 MSK

#### Жесткий дедлайн: 7.04.25 23:59 MSK

### Описание
Это творческое задание, в котором вам необходимо реализовать полный цикл построения рекомендательной системы: реализовать кандидат генераторов, придумать и собрать признаки, обучить итоговый ранкер и заинференсить модели на всех пользователей.

Вам предоставляется два набора данных: `train.csv` и `test.csv` 

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import implicit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import xgboost as xgb
from sklearn.model_selection import train_test_split
import shap

d:\HSE\Recsis_1(3-4)\DZ3\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# скачиваем данные
# если из этой ячейки не получается, то вот ссылка на папку https://drive.google.com/drive/folders/1HT0Apm8Jft0VPLJtdBBUGu9s1M7vZcoJ?usp=drive_link

!pip3 install gdown


import gdown
# train
url = "https://drive.google.com/uc?id=1-CcS22-UpTJeNcFlA0dVLrEQn8jnI0d-"

output = 'train.csv'
gdown.download(url, output, quiet=True)

# test
url = "https://drive.google.com/uc?id=11iz3xDh0IIoEIBY0dyRSvByY3qfiT3BG"

output = 'test.csv'
gdown.download(url, output, quiet=True)

# user features
url = "https://drive.google.com/uc?id=1zl2jWMdUhc-IMakHlihQhJ5PGGZm9-_O"
output = 'users.csv'
gdown.download(url, output, quiet=True, fuzzy=True)

# item features
url = "https://drive.google.com/uc?id=1chCmpiCKJRjdqNftHc-t2ALl3qbAp2G8"
output = 'items.csv'
gdown.download(url, output, quiet=True)

Downloading...
From: https://drive.google.com/file/d/1-CcS22-UpTJeNcFlA0dVLrEQn8jnI0d-/view?usp=drive_link
To: d:\HSE\Recsis_1(3-4)\DZ3\train.csv
91.9kB [00:00, 3.72MB/s]
Downloading...
From: https://drive.google.com/file/d/11iz3xDh0IIoEIBY0dyRSvByY3qfiT3BG/view?usp=drive_link
To: d:\HSE\Recsis_1(3-4)\DZ3\test.csv
92.1kB [00:00, 3.41MB/s]


'test.csv'

In [2]:
users_df = pd.read_csv('users.csv')
items_df = pd.read_csv('items.csv')



### 1 Этап. Модели первого уровня. (max 3 балла)
В этом этапе вам необходимо разделить `train` датасет на 2 части: для обучения моделей первого уровня и для их валидации. Единственное условие для разбиения – разбивать нужно по времени. Данные для обучение будем называть `train_stage_1`, данные для валидации `valid_stage_1`. Объемы этих датасетов вы определяет самостоятельно. 

Для начала нам нужно отобрать кандидатов при помощи легких моделей. Необходимо реализовать 3 типа моделей:
1. Любая эвристическая(алгоритмичная) модель на ваш выбор **(0.5 балл)**
2. Любая матричная факторизация на ваш выбор **(1 балл)**
3. Любая нейросетевая модель на ваш выбор **(1 балла)**

Не забудьте использовать скор каждой модели, как признак!



Каждая модель должна уметь:
1) для пары user_item предсказывать скор релевантности (масштаб скора не важен), важно обработать случаи, когда модель не можеn проскорить пользователя или айтем, вместо этого вернуть какое-то дефолтное значение
2) для всех пользователей вернуть top-k самых релевантных айтемов (тут вам скоры не нужны)


Дополнительно можно провести анализ кандидат генератов, измерить насколько различные айтемы они рекомендуют, например с помощью таких метрик как: [Ranked based overlap](https://github.com/changyaochen/rbo) или различные вариации [Diversity](https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation/blob/master/Base/Evaluation/metrics.py#L289). **(1 балл)**

In [3]:
# Загрузка данных
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
train_df['last_watch_dt'] = pd.to_datetime(train_df.last_watch_dt)
test_df['last_watch_dt'] = pd.to_datetime(test_df.last_watch_dt)

In [5]:
train_df['last_watch_dt'].max()

Timestamp('2021-08-12 00:00:00')

In [6]:
# Преобразование временных меток
train_df['weekday'] = train_df.last_watch_dt.dt.weekday
test_df['weekday'] = test_df.last_watch_dt.dt.weekday
train_df = train_df.sort_values('last_watch_dt')
test_df = test_df.sort_values('last_watch_dt')


train_stage_1 = train_df.loc[(train_df.last_watch_dt < '2021-08-06')].copy()
valid_stage_1 = train_df.loc[train_df.last_watch_dt >= '2021-08-06'].copy()

user_ids = train_df['user_id'].unique()
item_ids = train_df['item_id'].unique()
user_to_idx = {user: idx for idx, user in enumerate(user_ids)}
idx_to_user = {idx: user for user, idx in user_to_idx.items()}
item_to_idx = {item: idx for idx, item in enumerate(item_ids)}
idx_to_item = {idx: item for item, idx in item_to_idx.items()}

In [7]:
test_df.shape, train_stage_1.shape, valid_stage_1.shape

((608467, 7), (4473822, 7), (393134, 7))

In [25]:
# 1. Эвристическая модель (популярность по дням недели)
class HeuristicModel:
    def __init__(self):
        self.popularity = defaultdict(dict)
        self.default_score = 0.5
        
    def fit(self, df):
        for weekday in range(7):
            weekday_data = df[df['weekday'] == weekday]
            item_counts = weekday_data['item_id'].value_counts()
            total = item_counts.sum()
            for item, count in item_counts.items():
                self.popularity[weekday][item] = count / total
    
    def predict(self, user_id, item_id, weekday=None, default=None):
        if weekday is None:
            weekday = np.random.randint(0, 7)
            
        if item_id in self.popularity[weekday]:
            return self.popularity[weekday][item_id]
        return default if default is not None else self.default_score
    
    def recommend(self, user_id, k=10, weekday=None):
        if weekday is None:
            weekday = np.random.randint(0, 7)
            
        popular_items = sorted(self.popularity[weekday].items(), 
                             key=lambda x: -x[1])
        return [item for item, score in popular_items[:k]]

In [26]:
# 2. Матричная факторизация (implicit ALS)
class ImplicitFactorizationModel:
    def __init__(self, factors=50, iterations=15, regularization=0.01):
        self.model = implicit.als.AlternatingLeastSquares(
            factors=factors,
            iterations=iterations,
            regularization=regularization,
            random_state=42
        )
        self.default_score = 0.5
        self.user_factors = None
        self.item_factors = None
        
    def fit(self, df):
        # Создаем разреженную матрицу взаимодействий
        user_indices = df['user_id'].map(user_to_idx).values
        item_indices = df['item_id'].map(item_to_idx).values
        durations = df['total_dur'].values
        
        # Нормализуем продолжительности
        durations = durations / durations.max()
        
        # Создаем матрицу в формате COO
        from scipy.sparse import coo_matrix
        interactions = coo_matrix(
            (durations, (user_indices, item_indices)),
            shape=(len(user_to_idx), len(item_to_idx))
        )
        
        # Обучаем модель
        self.model.fit(interactions)
        self.user_factors = self.model.user_factors
        self.item_factors = self.model.item_factors
        
    def predict(self, user_id, item_id, default=None):
        if user_id not in user_to_idx or item_id not in item_to_idx:
            return default if default is not None else self.default_score
            
        user_idx = user_to_idx[user_id]
        item_idx = item_to_idx[item_id]
        return self.user_factors[user_idx] @ self.item_factors[item_idx].T
    
    def recommend(self, user_id, k=10):
        if user_id not in user_to_idx:
            return []
            
        user_idx = user_to_idx[user_id]
        items, scores = self.model.recommend(
            user_idx, 
            user_items=None, 
            N=k, 
            filter_already_liked_items=False
        )
        return [idx_to_item[item] for item in items]

In [27]:
# 3. Нейросетевая модель
class InteractionDataset(Dataset):
    def __init__(self, df, user_to_idx, item_to_idx):
        self.df = df.copy()
        self.df['user_idx'] = self.df['user_id'].map(user_to_idx)
        self.df['item_idx'] = self.df['item_id'].map(item_to_idx)
        self.df = self.df.dropna(subset=['user_idx', 'item_idx'])
        
        # Нормализация target
        self.scaler = MinMaxScaler()
        self.df['target'] = self.scaler.fit_transform(
            self.df[['total_dur']]
        )
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return (
            torch.LongTensor([row['user_idx']]),
            torch.LongTensor([row['item_idx']]),
            torch.FloatTensor([row['target']])
        )
    
class NeuralCF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim=32):
        super().__init__()
        self.user_embedding = nn.Embedding(n_users, emb_dim)
        self.item_embedding = nn.Embedding(n_items, emb_dim)
        self.fc = nn.Sequential(
            nn.Linear(emb_dim * 2, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, user, item):
        user_emb = self.user_embedding(user).squeeze(1)
        item_emb = self.item_embedding(item).squeeze(1)
        x = torch.cat([user_emb, item_emb], dim=1)
        return self.fc(x)

class TorchNeuralModel:
    def __init__(self, emb_dim=32, lr=0.01, batch_size=1024, n_epochs=3):
        self.model = None
        self.emb_dim = emb_dim
        self.lr = lr
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.default_score = 0.5
        self.scaler = MinMaxScaler()
        
    def fit(self, df):
        # Подготовка данных
        dataset = InteractionDataset(df, user_to_idx, item_to_idx)
        self.scaler = dataset.scaler
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        
        # Инициализация модели
        n_users = len(user_to_idx)
        n_items = len(item_to_idx)
        self.model = NeuralCF(n_users, n_items, self.emb_dim)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        
        # Обучение
        for epoch in range(self.n_epochs):
            for user, item, target in tqdm(dataloader, desc=f'Epoch {epoch+1}/{self.n_epochs}', leave=False):
                optimizer.zero_grad()
                output = self.model(user, item)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
    
    def predict(self, user_id, item_id, default=None):
        if user_id not in user_to_idx or item_id not in item_to_idx:
            return default if default is not None else self.default_score
            
        user_idx = torch.LongTensor([user_to_idx[user_id]])
        item_idx = torch.LongTensor([item_to_idx[item_id]])
        with torch.no_grad():
            score = self.model(user_idx, item_idx).item()
        return score
    
    def recommend(self, user_id, k=10):
        if user_id not in user_to_idx:
            return []
            
        user_idx = user_to_idx[user_id]
        user_tensor = torch.LongTensor([user_idx] * len(item_to_idx))
        item_tensor = torch.LongTensor(list(item_to_idx.values()))
        
        with torch.no_grad():
            scores = self.model(user_tensor, item_tensor).flatten().numpy()
        
        top_items = np.argsort(-scores)[:k]
        return [idx_to_item[item] for item in top_items]

In [31]:
# Инициализация и обучение моделей
print("Training Heuristic Model...")
heuristic_model = HeuristicModel()
heuristic_model.fit(train_stage_1)
print("Training Implicit ALS Model...")
implicit_model = ImplicitFactorizationModel(factors=50, iterations=15)
implicit_model.fit(train_stage_1)
print("Training PyTorch Neural Model...")
torch_model = TorchNeuralModel(emb_dim=32, n_epochs=3, batch_size=1024)
torch_model.fit(train_stage_1)

Training Heuristic Model...
Training Implicit ALS Model...


d:\HSE\Recsis_1(3-4)\DZ3\.venv\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.31058335304260254 seconds
  warnings.warn(
100%|██████████| 15/15 [00:53<00:00,  3.59s/it]


Training PyTorch Neural Model...


In [33]:
# Функция для оценки разнообразия рекомендаций
def evaluate_diversity(models, users, k=10):
    overlaps = np.zeros((len(models), len(models)))
    diversities = []
    
    for user in users[:1000]: 
        recommendations = []
        for model in models:
            try:
                recs = set(model.recommend(user, k=k))
                recommendations.append(recs)
            except:
                recommendations.append(set())
        
        # Рассчитываем попарные пересечения
        for i in range(len(models)):
            for j in range(len(models)):
                overlaps[i,j] += len(recommendations[i] & recommendations[j]) / k
                
        # Рассчитываем diversity для одного пользователя
        all_recs = set()
        for rec in recommendations:
            all_recs.update(rec)
        diversity = len(all_recs) / (len(models) * k)
        diversities.append(diversity)
    
    overlaps /= min(1000, len(users))
    mean_diversity = np.mean(diversities)
    
    print("Pairwise Overlap Matrix:")
    print(overlaps)
    print(f"\nMean Diversity: {mean_diversity:.3f}")
    
    return overlaps, mean_diversity

# Оценка разнообразия рекомендаций
print("\nEvaluating Recommendation Diversity...")
models = [heuristic_model, implicit_model, torch_model]
sample_users = valid_stage_1['user_id'].unique()[:1000]
overlaps, diversity = evaluate_diversity(models, sample_users)


Evaluating Recommendation Diversity...
Pairwise Overlap Matrix:
[[1.000e+00 2.156e-01 0.000e+00]
 [2.156e-01 1.000e+00 2.000e-04]
 [0.000e+00 2.000e-04 1.000e+00]]

Mean Diversity: 0.928



### 2 Этап. Генерация и сборка признаков. (max 2 балла)
Необходимо собрать минимум 10 осмысленных (`np.radndom.rand()` не подойдет) признаков, при этом:
1. 2 должны относиться только к сущности "пользователь" (например средний % просмотра фильмов у этой возрастной категории)
2. 2 должны относиться только к сущности "айтем" (например средний средний % просмотра данного фильма)
3. 6 признаков, которые показывают связь пользователя и айтема (например средний % просмотра фильмов с данным актером (айтем) у пользователей с таким же полом (пользователь)). 

### ВАЖНО!  

1. **В датасете есть колонка `watched_prct`. Ее можно использовать для генерации признаков (например сколько пользователь в среднем смотрит фильмы), но нельзя подавать в модель, как отдельную фичу, потому что она напрямую связана с target.**
2. **Все признаки должны быть собраны без дата лика, то есть если пользователь посмотрел фильм 10 августа, то признаки мы можем считать только на данных до 9 августа включительно.**


### Разбалловка
Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`. Поэтому есть 2 варианта сборки признаков, **реализовать нужно только 1 из них:**
1. Для обучения собираем признаки на первый день `valid_stage_1`, а для теста на первый день `test`. Например, если `valid_stage_1` начинается 5 сентября, то все признаки мы можем собирать только по 4 сентября включительно. **(1 балл)**
2. Признаки будем собирать честно на каждый день, то есть на 5 сентября собираем с начала до 4, на 6 сентября с начала до 5 и т.д. **(2 балла)**

In [8]:
# Сначала подготовим основные таблицы
train_df['last_watch_dt'] = pd.to_datetime(train_df['last_watch_dt'])
train_df = train_df.sort_values('last_watch_dt')

# Создадим функции для генерации признаков
def generate_features(df, users_df, items_df, date_cutoff):
    df_copy = df.copy()
    history = df[df['last_watch_dt'] < date_cutoff]

    # Признаки пользователя
    # 1. Средний % досмотра у пользователя
    user_avg_watch = history.groupby('user_id')['watched_pct'].mean().rename('user_avg_watch')
    # 2. Количество просмотров у пользователя
    user_watch_count = history.groupby('user_id')['item_id'].count().rename('user_watch_count')

    # Признаки айтема
    # 1. Средний % досмотра у айтема
    item_avg_watch = history.groupby('item_id')['watched_pct'].mean().rename('item_avg_watch')
    # 2. Количество просмотров у айтема
    item_watch_count = history.groupby('item_id')['user_id'].count().rename('item_watch_count')

    # Признаки взаимодействия

    # 1. Средний % досмотра для жанров, которые смотрел пользователь
    items_genre = items_df[['item_id', 'genres']].dropna()
    items_genre['genres'] = items_genre['genres'].apply(lambda x: x.split(',')[0])  # возьмем только первый жанр
    history = history.merge(items_genre, on='item_id', how='left')
    user_genre_avg = history.groupby(['user_id', 'genres'])['watched_pct'].mean().rename('user_genre_avg').reset_index()

    # 2. Средний % досмотра пользователями с тем же полом на этот жанр
    history = history.merge(users_df[['user_id', 'sex']], on='user_id', how='left')
    genre_sex_avg = history.groupby(['genres', 'sex'])['watched_pct'].mean().rename('genre_sex_avg').reset_index()

    # 3. Средний % досмотра по странам (item)
    items_country = items_df[['item_id', 'countries']].dropna()
    items_country['countries'] = items_country['countries'].apply(lambda x: x.split(',')[0])
    history = history.merge(items_country, on='item_id', how='left')
    user_country_avg = history.groupby(['user_id', 'countries'])['watched_pct'].mean().rename('user_country_avg').reset_index()

    # 4. Средний % досмотра по типу контента (movie/show)
    items_type = items_df[['item_id', 'content_type']]
    history = history.merge(items_type, on='item_id', how='left')
    user_content_type_avg = history.groupby(['user_id', 'content_type'])['watched_pct'].mean().rename('user_content_type_avg').reset_index()

    # 5. Средний % досмотра для фильмов, выпущенных до 2010 у пользователя
    items_year = items_df[['item_id', 'release_year']]
    history = history.merge(items_year, on='item_id', how='left')
    history['old_movie'] = history['release_year'] < 2010
    user_old_movie_avg = history.groupby(['user_id', 'old_movie'])['watched_pct'].mean().rename('user_old_movie_avg').reset_index()

    # 6. Средний % досмотра фильмов для детей у пользователя
    items_forkids = items_df[['item_id', 'for_kids']]
    history = history.merge(items_forkids, on='item_id', how='left')
    user_forkids_avg = history.groupby(['user_id'])['for_kids'].mean().rename('user_forkids_avg')

    # Собираем валидационный сет на конкретную дату
    current_data = df[df['last_watch_dt'] == date_cutoff].copy()

    # Мержим все признаки
    current_data = current_data.merge(user_avg_watch, on='user_id', how='left')
    current_data = current_data.merge(user_watch_count, on='user_id', how='left')
    current_data = current_data.merge(item_avg_watch, on='item_id', how='left')
    current_data = current_data.merge(item_watch_count, on='item_id', how='left')
    current_data = current_data.merge(user_forkids_avg, on='user_id', how='left')

    current_data = current_data.merge(items_genre, on='item_id', how='left')
    current_data = current_data.merge(user_genre_avg, on=['user_id', 'genres'], how='left')
    current_data = current_data.merge(users_df[['user_id', 'sex']], on='user_id', how='left')
    current_data = current_data.merge(genre_sex_avg, on=['genres', 'sex'], how='left')
    current_data = current_data.merge(items_country, on='item_id', how='left')
    current_data = current_data.merge(user_country_avg, on=['user_id', 'countries'], how='left')
    current_data = current_data.merge(items_type, on='item_id', how='left')
    current_data = current_data.merge(user_content_type_avg, on=['user_id', 'content_type'], how='left')
    current_data = current_data.merge(items_year, on='item_id', how='left')
    current_data['old_movie'] = current_data['release_year'] < 2010
    current_data = current_data.merge(user_old_movie_avg, on=['user_id', 'old_movie'], how='left')

    return current_data


In [9]:
# Генерация признаков для train и test
def generate_features_for_dates(df, users_df, items_df):
    dates = df['last_watch_dt'].dt.date.unique()
    features = []
    for date in tqdm(dates):
        date = pd.Timestamp(date)
        featured = generate_features(df, users_df, items_df, date)
        features.append(featured)
    return pd.concat(features)
train_df_with_features = generate_features_for_dates(train_df, users_df, items_df)
test_df_with_features = generate_features_for_dates(test_df, users_df, items_df)

100%|██████████| 10/10 [00:14<00:00,  1.48s/it]


In [10]:
train_df_with_features.tail()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,target,weekday,user_avg_watch,user_watch_count,item_avg_watch,...,user_genre_avg,sex,genre_sex_avg,countries,user_country_avg,content_type,user_content_type_avg,release_year,old_movie,user_old_movie_avg
56492,235493,11756,2021-08-12,6499,100.0,1,3,44.666667,9.0,54.374329,...,51.0,Ж,57.461850,США,21.250000,film,44.666667,1997.0,True,16.500000
56493,134400,341,2021-08-12,417,8.0,0,3,39.333333,12.0,5.521054,...,NaN,NaN,NaN,Россия,46.444444,series,54.333333,2021.0,False,39.333333
56494,598090,15297,2021-08-12,12120,75.0,1,3,NaN,NaN,55.358940,...,NaN,Ж,45.714737,Россия,NaN,series,NaN,2021.0,False,NaN
56495,889540,14899,2021-08-12,5085,100.0,1,3,68.789474,19.0,33.878127,...,10.0,Ж,45.981279,США,47.000000,film,70.882353,2021.0,False,71.153846
56496,99479,3427,2021-08-12,5758,25.0,0,3,31.600000,5.0,40.007477,...,NaN,Ж,49.238777,Россия,31.600000,series,21.000000,2020.0,False,31.600000


In [11]:
test_df_with_features.tail()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,target,weekday,user_avg_watch,user_watch_count,item_avg_watch,...,user_genre_avg,sex,genre_sex_avg,countries,user_country_avg,content_type,user_content_type_avg,release_year,old_movie,user_old_movie_avg
71166,795768,13211,2021-08-22,77,1.0,0,6,NaN,NaN,31.210526,...,NaN,NaN,NaN,Россия,NaN,film,NaN,1997.0,True,NaN
71167,793910,6162,2021-08-22,322,6.0,0,6,68.666667,3.0,50.498048,...,100.000000,Ж,34.180118,Испания,NaN,film,68.666667,2013.0,False,100.000000
71168,418605,9151,2021-08-22,270,4.0,0,6,23.637931,58.0,56.454545,...,45.500000,Ж,37.972520,Великобритания,24.000000,film,23.941176,2016.0,False,23.111111
71169,54946,8242,2021-08-22,24,0.0,0,6,30.000000,12.0,19.898990,...,30.142857,Ж,34.180118,США,35.666667,film,32.727273,1991.0,True,61.250000
71170,804,14282,2021-08-22,4061,66.0,1,6,35.625000,8.0,36.337349,...,36.833333,М,41.381618,Россия,44.000000,film,35.625000,2009.0,True,36.833333


In [12]:
train_stage_with_features_1 = train_df_with_features.loc[(train_df_with_features.last_watch_dt < '2021-08-06')].copy()
valid_stage_with_features_1 = train_df_with_features.loc[train_df_with_features.last_watch_dt >= '2021-08-06'].copy()


### 3 Этап. Обучение финального ранкера (max 2 балла)
Собрав все признаки из этапа 2, добавив скоры моделей из этапа 1 для каждой пары пользователь-айтем (где это возможно), пришло время обучать ранкер. В качестве ранкера можно использовать либо [xgboost](https://xgboost.readthedocs.io/en/stable/) или [catboost](https://catboost.ai/). Обучать можно как `Classfier`, так и `Ranker`, выбираем то, что лучше сработает. Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`, которую мы до сих пор не трогали.  Заметьте, что у нас в тесте есть холодные пользователи – те, кого не было в train и активные – те, кто был в train. Возможно их стоит обработать по отдельности (а может и нет).  
(1 балл)

После получения лучшей модели надо посмотреть на важность признаков и [shap values](https://shap.readthedocs.io/en/latest/index.html), чтобы:
1. Интерпритировать признаки, которые вы собрали, насколько они полезные
2. Проверить наличие ликов – если важность фичи в 100 раз больше, чем у всех остальных, то явно что-то не то  

(1 балл)






In [ ]:
# Подготовка данных для обучения
def prepare_data(train_data, test_data):
    # Выбираем только нужные колонки (исключаем watched_pct и другие ликующие признаки)
    features = [col for col in train_data.columns if col not in ['user_id', 'item_id', 'last_watch_dt', 
                                                               'watched_pct', 'target', 'weekday']]
    
    X_train = train_data[features].fillna(0)
    y_train = train_data['target']
    
    X_test = test_data[features].fillna(0)
    y_test = test_data['target']
    
    return X_train, y_train, X_test, y_test, features

# Подготовка данных
X_train, y_train, X_valid, y_valid, feature_names = prepare_data(
    train_stage_with_features_1,
    valid_stage_with_features_1
)

# Разделение на холодных и теплых пользователей
def split_cold_warm(data, train_users):
    warm_mask = data['user_id'].isin(train_users)
    return data[warm_mask], data[~warm_mask]

train_users = train_stage_with_features_1['user_id'].unique()
valid_warm, valid_cold = split_cold_warm(valid_stage_with_features_1, train_users)

X_valid_warm = valid_warm[feature_names].fillna(0)
y_valid_warm = valid_warm['target']

# Обучение XGBoost Ranker
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_names)
dvalid = xgb.DMatrix(X_valid_warm, label=y_valid_warm, feature_names=feature_names)

params = {
    'objective': 'rank:ndcg',
    'eval_metric': 'ndcg@5',
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

evals = [(dtrain, 'train'), (dvalid, 'valid')]
model = xgb.train(params, dtrain, num_boost_round=1000, 
                 evals=evals, early_stopping_rounds=50, verbose_eval=50)

In [ ]:
# Оценка на тестовых данных
test_warm, test_cold = split_cold_warm(test_df_with_features, train_users)
X_test_warm = test_warm[feature_names].fillna(0)
y_test_warm = test_warm['target']

dtest = xgb.DMatrix(X_test_warm, feature_names=feature_names)
test_preds = model.predict(dtest)

# Метрика NDCG
print(f"NDCG@5 на тесте: {ndcg_score([y_test_warm], [test_preds], k=5)}")

# Анализ важности признаков
importance = model.get_score(importance_type='weight')
importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

print("\nТоп-10 важных признаков:")
for feat, score in importance[:10]:
    print(f"{feat}: {score}")

# SHAP анализ
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train.iloc[:1000])

shap.summary_plot(shap_values, X_train.iloc[:1000], feature_names=feature_names)

In [ ]:
# Стратегия для холодных пользователей - используем популярные айтемы
if len(test_cold) > 0:
    # Вычисляем популярность айтемов в трейне
    item_popularity = train_stage_with_features_1.groupby('item_id')['target'].mean()
    
    # Сортируем айтемы по популярности
    top_items = item_popularity.sort_values(ascending=False).head(100).index
    
    # Для каждого холодного пользователя рекомендуем топовые айтемы
    test_cold_recommendations = []
    for user_id in test_cold['user_id'].unique():
        for item_id in top_items:
            test_cold_recommendations.append({'user_id': user_id, 'item_id': item_id, 'pred': 1.0})
    
    test_cold_preds = pd.DataFrame(test_cold_recommendations)


### 4 Этап. Инференс лучшего ранкера (max 3 балла)

Теперь мы хотим построить рекомендации "на завтра", для этого нам нужно:

1. Обучить модели первого уровня на всех (train+test) данных (0.5 балла)
2. Для каждой модели первого уровня для каждого пользователя сгененировать N кандидатов (0.5 балла)
3. "Склеить" всех кандидатов для каждого пользователя (дубли выкинуть), посчитать скоры от всех моделей (0.5 балла)
4. Собрать фичи для ваших кандидатов (теперь можем считать признаки на всех данных) (0.5 балла)
5. Проскорить всех кандидатов бустингом и оставить k лучших (0.5 балла)
6. Посчитать разнообразие(Diversity) и построить график от Diversity(k) (0.5 балла)


Все гиперпараметры (N, k) определяете только Вы!

In [ ]:
# YOUR CODE HERE